## An ECB/CBC detection oracle
Now that you have ECB and CBC working:

Write a function to generate a random AES key; that's just 16 random bytes.

Write a function that encrypts data under an unknown key --- that is, a function that generates a random key and encrypts under it.

The function should look like:
```
encryption_oracle(your-input)
=> [MEANINGLESS JIBBER JABBER]
```
Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.

Now, have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). Use rand(2) to decide which to use.

Detect the block cipher mode the function is using each time. You should end up with a piece of code that, pointed at a block box that might be encrypting ECB or CBC, tells you which one is happening.

In [2]:
import os
import random
from Crypto.Cipher import AES

In [ ]:
def pkcs7_pad(data: bytes, block_size: int = 16) -> bytes:
    remainder = len(data) % block_size
    padding_len = block_size - remainder
    padding = bytes([padding_len]) * padding_len
    return data + padding

def encryption_oracle(plain_t: str):
    data = bytes(plain_t, 'utf-8')
    c_b_plain = pkcs7_pad(data)
    
    random_key = os.urandom(32)
    random_iv = os.urandom(16)
    random_mode = random.choice(['ECB', 'CBC'])

    if random_mode == 'ECB':
        cipher =  AES.new(random_key, AES.MODE_ECB)
        ciphertext = cipher.encrypt(c_b_plain)
    else:
        cipher = AES.new(random_key, AES.MODE_CBC, random_iv)
        ciphertext = cipher.encrypt(c_b_plain)
    return ciphertext

def detect_aes_mode(cipher_t: bytes):
    blocks = [cipher_t[i:i+16] for i in range(0,len(cipher_t),16)]
    
    size_blocks = len(blocks)
    size_unique_block = len(set(blocks))
    
    if size_blocks > size_unique_block:
        print("Its AES-ECB...")
        return
    else:
        print("Its AES-CBC...")
        return

plain_text = "wonderful world of indonesia"
cipher = encryption_oracle(plain_text)
print(cipher)
detect_aes_mode(cipher)

b'\xd7f\xf6\xdd1\x9b\x0b\xc7x\xd0/\xca>\xa0OU\xd7f\xf6\xdd1\x9b\x0b\xc7x\xd0/\xca>\xa0OU\xd7f\xf6\xdd1\x9b\x0b\xc7x\xd0/\xca>\xa0OU\xd7f\xf6\xdd1\x9b\x0b\xc7x\xd0/\xca>\xa0OU\x81\xeeK\xe6\x00\xd1w\x04\xe2\xd1\x02]\xd2\xc2\xe0\x04'
Its AES-ECB...
